In [1]:
import sys
import os 

os.environ["CONFIG_PATHS"] = "../../configs/small_model.yaml"
os.environ["CONFIG_OVERRIDES"] = 'game.moves_directory="../../data/moves_10"'
sys.path.append("../../src")

import matplotlib.pyplot as plt
import scipy.stats

from analysis import load_events, filter_events

In [2]:
events = load_events("../../data/2024-11-21_16-37-53-medicinal/logs_2024-11-21_17-07-24_223037.txt")

Start time: 1732236205.317719


In [3]:
game_result_events = filter_events(events, "game_result")
game_results = [event.params for event in game_result_events]

players = set()
for game_result in game_results:
    for player, _ in game_result:
        players.add(player)

print(len(game_results))

769


In [4]:
elos = {player: 1000 for player in players}

elo_K = 32.0
elo_base = 400.0

for game_result in game_results:
    game_players = [player_name for player_name, _ in game_result]
    Q_player = [10 ** (elos[player] / elo_base) for player in game_players]
    Q_total = sum(Q_player)
    E_player = [Q / Q_total for Q in Q_player]

    for i, (player, result) in enumerate(game_result):
        elos[player] += elo_K * (result - E_player[i])

In [5]:
elos

{'s510189': 885.4060425007206,
 's379394': 1005.9676382048762,
 's127919': 1268.7320825202555,
 's253555': 839.8942367741458}

In [6]:
for player, E in zip(game_players, E_player):
    print(player, "\t", E)

s379394 	 0.1459969584905987
s510189 	 0.05970858083132205
s127919 	 0.7390999215659786
s253555 	 0.05519453911210083


In [7]:
results = {}
for game_result in game_results:
    for player, result in game_result:
        results[player] = results.get(player, 0) + result

total_results = sum(results.values())
for player in results:
    results[player] = results[player] / total_results

results

{'s510189': 0.27384048547897705,
 's253555': 0.2636540962288687,
 's379394': 0.25325097529258783,
 's127919': 0.20925444299956655}